In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as an
import scipy
import sklearn
import gget

sc.settings.verbosity = 3  

# Load Data

In [2]:
fpath = "/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/raw.anndata.h5ad"

adata = sc.read_h5ad(fpath)
adata.var_names = adata.var['gene_name'].values
adata.obs['cell'] = adata.obs.index.copy()
sc.logging.print_memory_usage()

adata

Memory usage: current 1.99 GB, difference +1.99 GB


AnnData object with n_obs × n_vars = 10883 × 19393
    obs: 'n_genes', 'cell'
    var: 'gene_name', 'Chromosome', 'Start', 'End', 'Strand'

In [3]:
adata.var_names

Index(['ATAD3B', 'PRDM16', 'SKI', 'PEX14', 'PLCH2', 'SPSB1', 'HES3', 'PLEKHM2',
       'CA6', 'NMNAT1',
       ...
       'BPY2C', 'CDY2B', 'SRY', 'UTY', 'VCY', 'DAZ3', 'DAZ1', 'AMELY',
       'RBMY1E', 'CDY1B'],
      dtype='object', length=19393)

# Load Predicted Phases 

In [4]:
fpath = "/home/cstansbu/git_repositories/biolegend_barcodes/result/hashmap.csv"
df = pd.read_csv(fpath)
df = df.rename(columns={'phase' : 'pred_phase', 'barcode' : 'cell_id'})
print(f"{df.shape=}")

df = df.set_index('cell_id')

adata.obs = pd.concat([adata.obs, df], ignore_index=False, axis=1)
adata.obs.head()

df.shape=(10879, 5)


,n_genes,cell,G1,G2M,S,pred_phase
cell_id,,,,,,
AAACCAAAGGGTAGCA,16001,AAACCAAAGGGTAGCA,80.0,403.0,87.0,G2M
AAACCAAAGTAAGGGT,7864,AAACCAAAGTAAGGGT,18.0,5.0,7.0,G1
AAACCATTCAGGTAGG,15610,AAACCATTCAGGTAGG,43.0,183.0,59.0,G2M
AAACCATTCCAGCCCT,10505,AAACCATTCCAGCCCT,229.0,43.0,1872.0,S
AAACCATTCGTGACCG,8775,AAACCATTCGTGACCG,58.0,1.0,2.0,G1


In [5]:
adata.obs['pred_phase'].value_counts(normalize=True, dropna=False)

pred_phase
G1     0.619774
G2M    0.207847
S      0.172011
NaN    0.000368
Name: proportion, dtype: float64

# Load cell cycle annotations

In [6]:
fpath = "../resources/seurat_genes.csv"

df = pd.read_csv(fpath)
df = df[df['gene_name'].isin(adata.var_names)]
df = df.set_index('gene_name')
df.columns = [f'seurat_{x.split(" ")[0]}' for x in df.columns]
new_cols = df.columns

adata.var = pd.concat([adata.var, df], ignore_index=False, axis=1)
adata.var[new_cols] = np.where(adata.var[new_cols].isna(), False, adata.var[new_cols])

adata.var.columns

Index(['gene_name', 'Chromosome', 'Start', 'End', 'Strand', 'seurat_S',
       'seurat_G2M'],
      dtype='object')

In [7]:
"""KEGG GENES"""
fpath = "../resources/KEGG_HUMAN_CELL_CYCLE.txt"
kegg_genes = [x.strip() for x in open(fpath)]

adata.var['is_kegg'] = adata.var['gene_name'].isin(kegg_genes)

adata.var.columns

Index(['gene_name', 'Chromosome', 'Start', 'End', 'Strand', 'seurat_S',
       'seurat_G2M', 'is_kegg'],
      dtype='object')

In [8]:
"""whitfield genes"""
fpath = "../resources/whitfield_clean.csv"
df = pd.read_csv(fpath)
df = df[df['gene_name'].isin(adata.var_names)]
df = df.set_index('gene_name')
df.columns = [f'whitfield_{x.split(" ")[0]}' for x in df.columns]
new_cols = df.columns

adata.var = pd.concat([adata.var, df], ignore_index=False, axis=1)
adata.var[new_cols] = np.where(adata.var[new_cols].isna(), False, adata.var[new_cols])

adata.var.columns

Index(['gene_name', 'Chromosome', 'Start', 'End', 'Strand', 'seurat_S',
       'seurat_G2M', 'is_kegg', 'whitfield_G1/S', 'whitfield_G2',
       'whitfield_G2/M', 'whitfield_M/G1', 'whitfield_S'],
      dtype='object')

In [9]:
"""gene ontology"""
fpath = "/nfs/turbo/umms-indikar/shared/projects/cell_cycle/data/go_cell_cycle_genes/human_cell_cycle_genes.csv"
df = pd.read_csv(fpath)
print(f"{df.shape=}")
df = df[df['gene_name'].isin(adata.var_names)]
print(f"{df.shape=}")
df = df.set_index('gene_name')
df.columns = [f"GO_{x.split(' ')[0]}" for x in df.columns]
new_cols = df.columns

adata.var = pd.concat([adata.var, df], ignore_index=False, axis=1)
adata.var[new_cols] = np.where(adata.var[new_cols].isna(), False, adata.var[new_cols])

adata.var.columns

df.shape=(142, 7)
df.shape=(125, 7)


Index(['gene_name', 'Chromosome', 'Start', 'End', 'Strand', 'seurat_S',
       'seurat_G2M', 'is_kegg', 'whitfield_G1/S', 'whitfield_G2',
       'whitfield_G2/M', 'whitfield_M/G1', 'whitfield_S', 'GO_G1', 'GO_G1/S',
       'GO_G2', 'GO_G2/M', 'GO_M', 'GO_S'],
      dtype='object')

In [10]:
cell_cycle_cols = [
    'seurat_S',
    'seurat_G2M',
    'is_kegg',
    'whitfield_G1/S', 
    'whitfield_G2', 
    'whitfield_G2/M', 
    'whitfield_M/G1',
    'whitfield_S', 
    'GO_G1', 
    'GO_G1/S',
    'GO_G2',
    'GO_G2/M', 
    'GO_M',
    'GO_S',
]

adata.var['cell_cycle'] = adata.var[cell_cycle_cols].any(axis=1)
adata.var['cell_cycle'].value_counts()

cell_cycle
False    19166
True       227
Name: count, dtype: int64

# Gene set scoring

In [ ]:
for column in cell_cycle_cols + ['cell_cycle']:
    gene_list = adata.var[adata.var[column]]['gene_name'].to_list()
    print(f"{column} n genes: {len(gene_list)}")
    
    sc.tl.score_genes(
        adata, 
        gene_list,
        ctrl_size=50,
        score_name=f"{column}_score"
    )

print('done')

seurat_S n genes: 43
computing score 'seurat_S_score'
    finished: added
    'seurat_S_score', score of gene set (adata.obs).
    950 total control genes are used. (0:00:00)
seurat_G2M n genes: 54
computing score 'seurat_G2M_score'
    finished: added
    'seurat_G2M_score', score of gene set (adata.obs).
    1044 total control genes are used. (0:00:00)
is_kegg n genes: 97
computing score 'is_kegg_score'
    finished: added
    'is_kegg_score', score of gene set (adata.obs).
    1044 total control genes are used. (0:00:00)
whitfield_G1/S n genes: 12
computing score 'whitfield_G1/S_score'
    finished: added
    'whitfield_G1/S_score', score of gene set (adata.obs).
    350 total control genes are used. (0:00:00)
whitfield_G2 n genes: 5
computing score 'whitfield_G2_score'
    finished: added
    'whitfield_G2_score', score of gene set (adata.obs).
    200 total control genes are used. (0:00:00)
whitfield_G2/M n genes: 11
computing score 'whitfield_G2/M_score'
    finished: added
    '

# Basic QC

In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

# filtration
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=5)

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True,
)

adata

# Basic Preprocessing

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata, target_sum=1e4)
adata.layers["norm"] = adata.X.copy()

# Logarithmize the data
sc.pp.log1p(adata)
adata.layers["log_norm"] = adata.X.copy()

sc.pp.highly_variable_genes(adata)

sc.tl.pca(
    adata,
    mask_var='highly_variable',
    n_comps=15,
)

sc.pl.pca_variance_ratio(adata)

sc.pp.neighbors(adata)

In [ ]:
sc.logging.print_memory_usage()

In [ ]:
# break

# UMAP/Clustering

In [ ]:
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.2)
adata

In [ ]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 4, 4

sc.pl.umap(
    adata, 
    color=["leiden",
           "pred_phase",
           "total_counts",
           "n_genes",
          ],
    ncols=2,
)

In [ ]:
""" plot the reads associated with each biolegend feature """

adata.obs['pred_G1'] = np.log1p(adata.obs['G1'])
adata.obs['pred_S'] = np.log1p(adata.obs['S'])
adata.obs['pred_G2M'] = np.log1p(adata.obs['G2M'])

sc.pl.umap(
    adata, 
    color=["pred_phase", "pred_G1", "pred_S","pred_G2M"],
    ncols=2,
)

In [ ]:
sc.pl.umap(
    adata,
    color=[
        'seurat_S_score', 'seurat_G2M_score', 
        'whitfield_G1/S_score', 'whitfield_G2_score', 'whitfield_G2/M_score', 'whitfield_M/G1_score', 'whitfield_S_score', 
        'GO_G1_score', 'GO_G1/S_score', 'GO_G2_score', 'GO_G2/M_score', 'GO_M_score', 'GO_S_score',
    ],
    ncols=4,
)

# Gene Expression

In [ ]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 4, 4

sc.pl.umap(
    adata, 
    color=['APP', "PRIM1", "CDK4", "CDKN1A", "GMNN", "ABCB1"],
    ncols=3,
)

# DEG by predicted phase

In [ ]:
sc.tl.rank_genes_groups(
    adata, 
    groupby="pred_phase",
    method='wilcoxon',
    corr_method='benjamini-hochberg',
    pts=True,
)

deg = sc.get.rank_genes_groups_df(
    adata, 
    group=None
)

deg.head()

In [ ]:
logfoldchanges = 1.0
pvals_adj = 0.05
n_genes = 5

sig = deg.copy()

sig = sig[sig['logfoldchanges'] > logfoldchanges]
sig = sig[sig['pvals_adj'] < pvals_adj]

gx = sig.groupby('group').head(n_genes)

sc.pl.umap(
    adata,
    color=gx['names'].to_list(),
    ncols=n_genes,
)    

gx

In [ ]:
logfoldchanges = 0.0
pvals_adj = 0.05
n_genes = 5

sig = deg.copy()

sig = sig[sig['logfoldchanges'] > logfoldchanges]
sig = sig[sig['pvals_adj'] < pvals_adj]

cycle_genes = adata.var[adata.var['cell_cycle']]['gene_name'].values

sig = sig[sig['names'].isin(cycle_genes)]
gx = sig.groupby('group').head(n_genes)

sc.pl.umap(
    adata,
    color=gx['names'].to_list(),
    ncols=n_genes,
)

gx

In [ ]:
columns = [
    'seurat_S','seurat_G2M',
    'whitfield_G1/S', 'whitfield_G2','whitfield_G2/M', 'whitfield_M/G1', 'whitfield_S', 
    'GO_G1', 'GO_G1/S','GO_G2', 'GO_G2/M', 'GO_M', 'GO_S',
]

adata.var[adata.var['gene_name'].isin(gx['names'])][columns]

In [ ]:
logfoldchanges = 1.0
pvals_adj = 0.05
n_genes = 25
database = 'ontology'

sig = deg.copy()
sig = sig[sig['logfoldchanges'] > logfoldchanges]
sig = sig[sig['pvals_adj'] < pvals_adj]
sig = sig.sort_values(by=['group', 'logfoldchanges'], ascending=[True, False])

# Informative print statements with formatting
for group_name, group_df in sig.groupby('group'):
    print(f"\n{'='*20} Analyzing group: {group_name} {'='*20}")  # Clear group header
    
    group_df = group_df.head(n_genes)
    query = group_df['names'].to_list()
    edf = gget.enrichr(query, database)
    edf = edf[['path_name', 'adj_p_val', 'overlapping_genes']]

    if not edf.empty:
        print(f"\nTop Enrichment Results for '{group_name}':\n")
        print(edf.head(10).to_string(index=False))  # Nicely formatted table
    else:
        print(f"\nNo significant enrichment results found for '{group_name}'.\n")

    print("-" * 50)  # Separator between groups

# DPT

In [ ]:
adata.uns['iroot'] = np.flatnonzero(adata.obs['pred_phase'] == 'G1')[0]
sc.tl.dpt(adata)

sc.pl.umap(
    adata, 
    color=['pred_phase', 'dpt_pseudotime']
)

In [ ]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 4, 3

colors = [
    'r', 'green', 'gold',
]

sns.boxplot(
    data=adata.obs[adata.obs['leiden'].isin(['0', '1'])],
    x='pred_phase',
    y='dpt_pseudotime',
    hue='pred_phase',
    order=['G1', 'S', 'G2M'],
    palette=colors,
    showfliers=False,
    width=0.5,
)

plt.ylabel("Pseudotime")
plt.xlabel("")
sns.despine()

# write the object out

In [ ]:
out_path = "/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/processed.anndata.h5ad"


for column in adata.var.columns:
    if adata.var[column].dtype == 'object':
        adata.var[column] = adata.var[column].astype(str)

# write the object to file
adata.write(out_path)

adata

In [ ]:
break